In [1]:
%pip install qutip
%pip install qutip_qip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 42.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 KB 8.7 MB/s eta 0:00:00


# Quantum Teleportation



Quantum teleportation is a process in quantum communication where the state of an unknown quantum particle is transferred from one location to another, without actually transmitting the particle itself. The process makes use of the principles of quantum entanglement and quantum measurement to achieve this feat.

Mathematically, quantum teleportation can be described as follows:

Consider two qubits, A and B, in a maximally entangled state described by the following wave function:

$$\begin{equation}
\frac{1}{\sqrt{2}} \left(\left|00\right\rangle + \left|11\right\rangle\right)
\end{equation}$$

Let the unknown state of qubit A be described by the wave function $$\left|\psi\right\rangle = \alpha\left|0\right\rangle + \beta\left|1\right\rangle$$.

The first step in quantum teleportation is to perform a Bell basis measurement on qubits A and B. The Bell basis measurement projects the two qubits onto the following four states:
$$
\begin{equation}
\left|\Phi^\pm\right\rangle = \frac{1}{\sqrt{2}} \left(\left|00\right\rangle \pm \left|11\right\rangle\right) \
\left|\Psi^\pm\right\rangle = \frac{1}{\sqrt{2}} \left(\left|01\right\rangle \pm \left|10\right\rangle\right)
\end{equation}$$

The result of the measurement is then transmitted to the recipient through classical communication.

Based on the measurement result, the recipient can then perform a series of single-qubit rotations on qubit B to obtain the desired state $\left|\psi\right\rangle$.

The mathematical insight behind quantum teleportation lies in the fact that it demonstrates the nonlocal behavior of quantum systems and the presence of entanglement. The ability to transfer the state of an unknown quantum particle from one location to another without actually transmitting the particle itself is a remarkable result that has implications in various areas of quantum communication and quantum computing.


In [2]:
from qutip_qip.circuit import QubitCircuit, Measurement, Gate
from qutip import basis, tensor, identity
from math import sqrt

## Introduction

This notebook introduces to the basics of quantum teleportation

In [3]:
teleportation = QubitCircuit(3, num_cbits = 2, input_states = ["\psi", "0", "0", "c0", "c1"])


First, Alice and Bob need to create the shared EPR pair $\frac{| 00 ⟩ + | 11 ⟩} {2}
$ between the second and third qubit by using the hadamard gate on Alice's qubit followed by an entangling CNOT gate.

In [4]:
teleportation.add_gate("SNOT", targets=[1])
teleportation.add_gate("CNOT", targets=[2], controls=[1])

Following this, Alice makes the qubit $|q0⟩$
 interact with Alice's EPR qubit, followed by measuring on the two qubits belonging to Alice. The measurement results for the first qubit is stored in classical register $c_1$
 and the second qubit is stored in classical register $c_0$
.

In [5]:
teleportation.add_gate("CNOT", targets=[1], controls=[0])
teleportation.add_gate("SNOT", targets=[0])

teleportation.add_measurement("M0", targets=[0], classical_store=1)
teleportation.add_measurement("M1", targets=[1], classical_store=0)

Now, we apply the X
 gate on Bob's qubit based on the classical control $c_0$
 and $Z$
 gate based on classical control $c_1$
. These operations correspond to the following operations based on the state of Alice's measurement.

$$|00⟩→
 \text{no operation}$$
$$|01⟩→Z$$

$$|10⟩→X$$

$$|11⟩→ZX$$
The final circuit mathematically must result in the third qubit taking the state $|ψ⟩$
.

In [6]:
teleportation.add_gate("X", targets=[2], classical_controls=[0])
teleportation.add_gate("Z", targets=[2], classical_controls=[1])

Finally, our teleportation circuit is ready to run, we can view the circuit structure using the following command.

In [7]:
teleportation.gates

[Gate(SNOT, targets=[1], controls=None, classical controls=None, control_value=None),
 Gate(CNOT, targets=[2], controls=[1], classical controls=None, control_value=None),
 Gate(CNOT, targets=[1], controls=[0], classical controls=None, control_value=None),
 Gate(SNOT, targets=[0], controls=None, classical controls=None, control_value=None),
 Measurement(M0, target=[0], classical_store=1),
 Measurement(M1, target=[1], classical_store=0),
 Gate(X, targets=[2], controls=None, classical controls=[0], control_value=None),
 Gate(Z, targets=[2], controls=None, classical controls=[1], control_value=None)]

In [8]:
teleportation

### Example 1:

$$|\psi⟩ = |+⟩$$

In [9]:
a = 1/sqrt(2) * basis(2, 0) + 1/sqrt(2) * basis(2, 1)
state = tensor(a, basis(2,0), basis(2,0))

We can confirm our state is initialized correctly by observing the measurment statistics on the first qubit, followed by which we run the circuit.

In [10]:
initial_measurement = Measurement("start", targets=[0])
initial_measurement.measurement_comp_basis(state)

([Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[1.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]],
  Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[0.]
   [0.]
   [0.]
   [0.]
   [1.]
   [0.]
   [0.]
   [0.]]],
 [0.4999999999999999, 0.4999999999999999])

We can run the circuit using the `QubitCircuit.run()` function which provided the initial state-vector (or density matrix) initiates one run on the circuit (including sampling any intermediate measurements) and providing the final results (any classical bits can also be explicitly set using the argument `cbits`). The results are returned as a Result object. The result states can be accessed through the `get_states()` function where the argument index=0 specifies the first(only) result should be returned

In [11]:
state_final = teleportation.run(state)
print(state_final)

Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]
 [0.        ]]


After running, we can see the measurement statistics on the last qubit to see that the qubit is teleported correctly.

In [12]:
final_measurement = Measurement("start", targets=[2])
final_measurement.measurement_comp_basis(state_final)

([Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[0.]
   [0.]
   [0.]
   [0.]
   [1.]
   [0.]
   [0.]
   [0.]],
  Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [1.]
   [0.]
   [0.]]],
 [0.4999999999999999, 0.4999999999999999])

## Example : 2
$$|\psi⟩ = |1⟩$$

In [13]:
state = tensor(basis(2,1), basis(2,0), basis(2,0))
initial_measurement = Measurement("start", targets=[0])
initial_measurement.measurement_comp_basis(state)

([None,
  Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[0.]
   [0.]
   [0.]
   [0.]
   [1.]
   [0.]
   [0.]
   [0.]]],
 [0.0, 1.0])

In [14]:
state_final = teleportation.run(state)
print(state_final)

Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [15]:
final_measurement = Measurement("start", targets=[2])
final_measurement.measurement_comp_basis(state_final)

([None,
  Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
  Qobj data =
  [[0.]
   [1.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]]],
 [0.0, 1.0])

Another useful feature of the circuit module is the `QubitCircuit.run_statistics()` feature which provides the opportunity to gather all the possible output states of the circuit along with their output probabilities. Again, the results are returned as a `Result` object. The result states and respective probabilites can be accessed through the `get_results()` function.

In [16]:
results = teleportation.run_statistics(state)
results.probabilities

[0.24999999999999994,
 0.24999999999999994,
 0.24999999999999994,
 0.24999999999999994]

In [17]:
results.final_states

[Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
 Qobj data =
 [[0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]],
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]],
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]],
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]]]

### Software Versions

In [18]:
from qutip.ipynbtools import version_table
version_table()

Software,Version
QuTiP,4.7.1
Numpy,1.21.6
SciPy,1.7.3
matplotlib,3.2.2
Cython,0.29.33
Number of CPUs,2
BLAS Info,OPENBLAS
IPython,7.9.0
Python,"3.8.10 (default, Nov 14 2022, 12:59:47) [GCC 9.4.0]"
OS,posix [linux]
